### Ch.15 RNN과 CNN을 이용한 Sequence Data 분석-1 (Short Sequence)

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layer
import numpy as np

1. 우리가 사용하는 많은 데이터는 DNN이나 CNN으로는 분석이 불가능한 연속적인 sequence data이다. 예를 들면 음성이나 이미지, 동영상 등 모두 이전의 데이터를 이용해서 이후의 데이터를 분석하는 방법을 사용하게 되는데, 이에 사용이 가능한 것이 순환 신경망인 RNN (Recurrent Neural Network)이다.
    - 이 신경망에서는 time series 데이터를 분석해서 주식가격을 예측할 수 있고, 자율 주행 시스템에서 차의 이동 경로를 예측해 줄 수 있으며, 문장, 문서, 오디오 샘플 분석, 자동 번역, text-to-speech같은 자연어 처리에 유용하다.
    
2. RNN은 일반적으로 이전의 상태가 다음의 상태에 영향을 미치게 된다.
    - 벡터 X의 시퀀스 데이터를 매 time step마다 recurrence formula를 적용하는 방법으로 분석하게 된다.
    - H(t) = Fw( H(t-1), X(t) )라는 방법으로 분석이 된다.
        - H(t)는 새로운 상태, H(t-1)는 이전의 상태, Fw는 가중치 w를 가지는 함수, X(t)는 특정 timestep의 input vector(특정 시간의 상태)이다.
    - 모든 RNN cell에 적용이 되는 Fw가, 즉 가중치가 동일하기 때문에 cell이 한 층에 많음에도 불구하고 일반적으로는 1개로 표현을 하곤한다.
    - 그리고 최종적으로 output layer에 출력되는 값은 W_hy가중치를 곱해서 계산이 되는데,  
        - 결국 H(t)는 tanh(W_hh * H(t-1) + W_xh * X(t))이다. 즉, 이전의 값에 가중치를 곱하고 현재 입력 값에 가중치를 곱한뒤 더해서 활성화 함수에 입력되는 값이다.  

3. 정리하자면 매 time step, 혹은 frame마다 순환 뉴런(recurrent neuron)은 X(t)뿐만 아니라 이전 타임 스텝의 출력인 H(t-1)을 받고, 각각의 뉴런은 이에 대한 가중치 **한 쌍**을 가지고 있다. 이 값들은 뉴런이 한 개가 아닌 이상 가중치 행렬 Wx, Wy로 바꿀 수 있으며, 뿐만 아니라 타임 스텝 t에서의 모든 입력을 행렬 X(t)로 만들어 mini batch 전체에 대해서 순환층의 출력을 한 번에 계산이 가능하게 된다.  

4. 
- Y(t)는 (미니 배치의 샘플 수 * 뉴런의 수)인 행렬이 된다. 
- X(t)는 (미니 배치의 샘플 수 * 입력 특성의 수)
- Wx, 즉 X(t)에 곱하는 가중치는 (입력 특성의 수 * 뉴런의 수)
- Wy, 즉 Y(t)에 곱하는 가중치는 (뉴런의 수 * 뉴런의 수)
- b, 즉 각 뉴련의 편향은 (뉴런의 수) 크기의 벡터이다.
   

**1. 간단하게 'hello'라는 단어를 입력해 다음 단어를 예측하는 RNN을 짜보고자 한다.**
1. input_size = 4, hidden_size = 2인 모델을 만들어 볼 것이다.
2. 문자를 벡터로 바꾸는 방법은 여러개가 있겠지만, 우선은 one-hot-encoding을 이용해서 벡터로 바꿀 것이다.
3. 밑의 코드에서 output은 나온 H(t)값이고 state는 옆으로 이동해서 영향을 줄 값이다.
4. hidden_size, 즉 출력의 크기가 2이기 때문에 출력 뉴런의 개수가 2라는 의미이고 즉 나온 output과 state의 각각의 값은 둘 다 다음 값을 각각의 뉴런이 예측한 값이다.

In [4]:
h,e,l,o = [1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]

x_data = np.array([[h]], dtype = np.float32)

hidden_size = 2
cell = layer.SimpleRNNCell(units = hidden_size)
rnn = layer.RNN(cell, return_sequences = True, return_state = True)
outputs, state = rnn(x_data)

print("x_data : {} x_data_shape : {}".format(x_data, x_data.shape))
print("output : {} output_shape : {}".format(outputs, outputs.shape))
print("state : {} state_shape : {}".format(state, state.shape))



x_data : [[[1. 0. 0. 0.]]] x_data_shape : (1, 1, 4)
output : [[[-0.02047562 -0.35498825]]] output_shape : (1, 1, 2)
state : [[-0.02047562 -0.35498825]] state_shape : (1, 2)


**2. 이번에는 간단한 many-to-one 시퀀스 네트워크를 구현해 보고자 한다.**
1. 문자열 시퀀스를 입력받을 때에는 일반적으로 문자열을 tokenization을 통해 분할하고 keras의 embedding layer을 이용해서 벡터화한 데이터를 RNN이 사용할 수 있도록 해준다.
2. 이번에는 단어를 입력하는 것이므로 토큰화를 한다면 글자 별로 하게 될 것이다.

In [9]:
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]

#토큰화한 문자에 대한 사전을 만든다.
token_dict = ['<pad>'] + sorted(list(set(''.join(words))))
char_2idx = {idx : char for idx, char in enumerate(token_dict)}
idx_2char = {char : idx for idx, char in enumerate(token_dict)}

print(token_dict)
print(idx_2char)
print(char_2idx)

['<pad>', ' ', 'a', 'b', 'd', 'e', 'g', 'o', 'r', 's', 'w']
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}
{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}


In [12]:
x_data = list(map(lambda x : [idx_2char.get(char) for char in x], words))
print(x_data)
max_sequence = 10

from tensorflow.keras.preprocessing.sequence import pad_sequences
#pad_sequence는 기본적으로 0으로 빈 자리를 padding해준다.
#이를 해주는 이유는 max_sequence의 길이인 10으로 모든 데이터의 길이를 통일 시켜 주어야만 RNN이 학습을 할 수 있기 때문이다.
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence, padding = 'post', truncating = 'post')
#pad_sequence에 넣은 parameter의 값들은 모두 뒤의 넘치는 값들을 없애 줄것이라는 의미이다.
#trunctaing은 넘치면 어떻게 할 것인지, padding은 모자라면 어떻게 할 것인지를 결정 짓게 하는 변수이다.
print(x_data)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]


```
tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs
)
```   
위의 내용이 원래 Embedding layer안에서 결정 지어 줄 수 있는 변수들이다.
1. embeddings_initializer: Initializer for the embeddings matrix ```keras.initializers```
2. embeddings_regularizer: Regularizer function applied to the embeddings matrix ```keras.regularizers```
3. embeddings_constraint: Constraint function applied to the embeddings matrix ```keras.constraints```

In [15]:
input_dim = len(char_2idx)
output_dim = len(char_2idx)

hidden_size = 10
num_classes = 2

#np.eye(n, dtype = int)같은 식으로 하면 3개만큼 [[1,0,0],[0,1,0],[0,0,1]] 이런식으로 해당 단어의 index값으로 one-hot-encoding을 한다.
one_hot = np.eye(len(char_2idx))
model = keras.models.Sequential()
model.add(layer.Embedding(input_dim = input_dim, output_dim = output_dim, trainable = False, mask_zero = True, 
                          input_length = max_sequence, embeddings_initializer = keras.initializers.Constant(one_hot)))
model.add(layer.SimpleRNN(units = hidden_size))
model.add(layer.Dense(units = num_classes))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 11)            121       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 10)                220       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22        
Total params: 363
Trainable params: 242
Non-trainable params: 121
_________________________________________________________________


In [27]:
#classification문제이기 떄문에 crossentropy를 사용하고, label인 y가 integer의 형태, 즉 one-hot이 아니기 떄문에 sparse를 사용한다.
def loss_fn(model,x,y):
    return keras.losses.sparse_categorical_crossentropy(y_true = y, y_pred = model(x), from_logits = True)

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.1)

dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).shuffle(4).batch(2)
dataset

<BatchDataset shapes: ((None, 10), (None,)), types: (tf.int32, tf.int32)>

In [30]:
losses = []
epochs = 30
for i in range(epochs):
    avg_train_loss = 0
    train_step = 0
    
    for x,y in dataset:
        with tf.GradientTape() as tape:
            loss = loss_fn(model, x, y)
        gradient = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(zip(gradient, model.variables))
        avg_train_loss += loss
        train_step += 1
        
    else:
        avg_train_loss /= train_step
        losses.append(avg_train_loss)
        
    if (i%5 == 0):
        print("Epoch : {} Loss : {}".format(i+1, avg_train_loss))

Epoch : 1 Loss : [0.6856068  0.56068444]
Epoch : 6 Loss : [6.204646e-05 6.019874e-05]
Epoch : 11 Loss : [6.6756979e-06 5.1855905e-06]
Epoch : 16 Loss : [2.9206233e-06 2.0861603e-06]
Epoch : 21 Loss : [1.7285331e-06 1.8477422e-06]
Epoch : 26 Loss : [1.9073468e-06 1.3113013e-06]


In [33]:
np.argmax(model.predict(x_data),axis = -1)

array([1, 0, 0, 1], dtype=int64)

**마지막 출력값만이 의미가 있는 것이기 때문에 axis = -1을 해서 argmax값을 구했고, 그 결과 우리가 원한 [1,0,0,1] 이 나왔다.**

#### 3. 간단한 many-to-one-stacked network를 이용해서 문장을 분석해 보고자 한다.

In [34]:
#1. dataset 만들기
sentences = ['When life offers you a dream so far beyond any of your expectations, it’s not reasonable to grieve when it comes to an end.',
            'I said it would be better if we weren’t friends, not that I didn’t want to be.',
            'And third, I was unconditionally and irrevocably in love with him.',
            'No matter how perfect the day is, it always has to end.']
y_label = [1,0,1,0] #1:Bella Swan 2:Edward Cullen

char_set = ['<pad>'] + sorted(list(set(''.join(sentences))))
char_2idx = {char : idx  for idx, char in enumerate(char_set)}
idx_2char = {idx : char for idx, char in enumerate(char_set)}

print(char_set)
print(char_2idx)
print(idx_2char)

['<pad>', ' ', ',', '.', 'A', 'I', 'N', 'W', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', '’']
{'<pad>': 0, ' ': 1, ',': 2, '.': 3, 'A': 4, 'I': 5, 'N': 6, 'W': 7, 'a': 8, 'b': 9, 'c': 10, 'd': 11, 'e': 12, 'f': 13, 'g': 14, 'h': 15, 'i': 16, 'l': 17, 'm': 18, 'n': 19, 'o': 20, 'p': 21, 'r': 22, 's': 23, 't': 24, 'u': 25, 'v': 26, 'w': 27, 'x': 28, 'y': 29, '’': 30}
{0: '<pad>', 1: ' ', 2: ',', 3: '.', 4: 'A', 5: 'I', 6: 'N', 7: 'W', 8: 'a', 9: 'b', 10: 'c', 11: 'd', 12: 'e', 13: 'f', 14: 'g', 15: 'h', 16: 'i', 17: 'l', 18: 'm', 19: 'n', 20: 'o', 21: 'p', 22: 'r', 23: 's', 24: 't', 25: 'u', 26: 'v', 27: 'w', 28: 'x', 29: 'y', 30: '’'}


In [37]:
x_data = list(map(lambda x: [char_2idx.get(word) for word in x], sentences))
x_datalen = list(map(lambda x: len(x), x_data))

print(x_data)
print(x_datalen)

[[7, 15, 12, 19, 1, 17, 16, 13, 12, 1, 20, 13, 13, 12, 22, 23, 1, 29, 20, 25, 1, 8, 1, 11, 22, 12, 8, 18, 1, 23, 20, 1, 13, 8, 22, 1, 9, 12, 29, 20, 19, 11, 1, 8, 19, 29, 1, 20, 13, 1, 29, 20, 25, 22, 1, 12, 28, 21, 12, 10, 24, 8, 24, 16, 20, 19, 23, 2, 1, 16, 24, 30, 23, 1, 19, 20, 24, 1, 22, 12, 8, 23, 20, 19, 8, 9, 17, 12, 1, 24, 20, 1, 14, 22, 16, 12, 26, 12, 1, 27, 15, 12, 19, 1, 16, 24, 1, 10, 20, 18, 12, 23, 1, 24, 20, 1, 8, 19, 1, 12, 19, 11, 3], [5, 1, 23, 8, 16, 11, 1, 16, 24, 1, 27, 20, 25, 17, 11, 1, 9, 12, 1, 9, 12, 24, 24, 12, 22, 1, 16, 13, 1, 27, 12, 1, 27, 12, 22, 12, 19, 30, 24, 1, 13, 22, 16, 12, 19, 11, 23, 2, 1, 19, 20, 24, 1, 24, 15, 8, 24, 1, 5, 1, 11, 16, 11, 19, 30, 24, 1, 27, 8, 19, 24, 1, 24, 20, 1, 9, 12, 3], [4, 19, 11, 1, 24, 15, 16, 22, 11, 2, 1, 5, 1, 27, 8, 23, 1, 25, 19, 10, 20, 19, 11, 16, 24, 16, 20, 19, 8, 17, 17, 29, 1, 8, 19, 11, 1, 16, 22, 22, 12, 26, 20, 10, 8, 9, 17, 29, 1, 16, 19, 1, 17, 20, 26, 12, 1, 27, 16, 24, 15, 1, 15, 16, 18, 3], [6, 20

In [39]:
max_sequence = 90
#빈자리나 남는 자리를 채워서 max_sequence에 맞게 입력 데이터를 만들기 위해 pad_sequence 사용
x_data = pad_sequences(x_data, max_sequence, padding = 'post', truncating = 'post')
print(x_data)


[[ 7 15 12 19  1 17 16 13 12  1 20 13 13 12 22 23  1 29 20 25  1  8  1 11
  22 12  8 18  1 23 20  1 13  8 22  1  9 12 29 20 19 11  1  8 19 29  1 20
  13  1 29 20 25 22  1 12 28 21 12 10 24  8 24 16 20 19 23  2  1 16 24 30
  23  1 19 20 24  1 22 12  8 23 20 19  8  9 17 12  1 24]
 [ 5  1 23  8 16 11  1 16 24  1 27 20 25 17 11  1  9 12  1  9 12 24 24 12
  22  1 16 13  1 27 12  1 27 12 22 12 19 30 24  1 13 22 16 12 19 11 23  2
   1 19 20 24  1 24 15  8 24  1  5  1 11 16 11 19 30 24  1 27  8 19 24  1
  24 20  1  9 12  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4 19 11  1 24 15 16 22 11  2  1  5  1 27  8 23  1 25 19 10 20 19 11 16
  24 16 20 19  8 17 17 29  1  8 19 11  1 16 22 22 12 26 20 10  8  9 17 29
   1 16 19  1 17 20 26 12  1 27 16 24 15  1 15 16 18  3  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 6 20  1 18  8 24 24 12 22  1 15 20 27  1 21 12 22 13 12 10 24  1 24 15
  12  1 11  8 29  1 16 23  2  1 16 24  1  8 17 27  8 29 23  1 15  8 23  1
  24 20  1 12 1

In [43]:
#model 만들기
num_classes = 2
hidden_dims = [10,10]

input_dim = len(char_2idx)
output_dim = len(char_2idx)
one_hot = np.eye(len(char_2idx))

model=keras.models.Sequential()
#mask_zero = True로 설정해 주는 방법으로 입력 데이터의 0은 무시해 준다.
model.add(keras.layers.Embedding(input_dim = input_dim, output_dim = output_dim, 
                                trainable = False, mask_zero = True, input_length = max_sequence,
                                embeddings_initializer = keras.initializers.Constant(one_hot)))
model.add(keras.layers.SimpleRNN(units = hidden_dims[0], return_sequences = True))
#TimeDistributed layer는 모든 타임 스텝에 Dropout을 적용하기 위해서 사용이 된다.
#이 층은 다른 층을 감싸서 입력 시퀀스의 모든 타임 스텝에 적용을 하게 한다.
model.add(keras.layers.TimeDistributed(layer.Dropout(0.3)))
model.add(keras.layers.SimpleRNN(units = hidden_dims[1]))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(units = num_classes))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 90, 31)            961       
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 90, 10)            420       
_________________________________________________________________
time_distributed (TimeDistri (None, 90, 10)            0         
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 10)                210       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 1,613
Trainable params: 652
Non-trainable params: 961
____________________________________________________

In [45]:
#model에 dropout layer이 있는데 이는 training 단계에서만 활용하기 때문에 이를 조절해 주기 위해서 training 변수를 loss_fn에 추가한다.
def loss_fn(model, x,y,training):
    return tf.compat.v1.losses.sparse_softmax_cross_entropy(labels=y, logits = model(x, training))

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.1)
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_label)).shuffle(4).batch(2)

dataset

<BatchDataset shapes: ((None, 90), (None,)), types: (tf.int32, tf.int32)>

In [46]:
epochs = 30
losses = []
for i in range(epochs):
    avg_loss = 0
    train_step = 0
    for x,y, in dataset:
        with tf.GradientTape() as tape:
            loss = loss_fn(model, x,y,training = True)
        gradient = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(zip(gradient, model.variables))
        avg_loss += loss
        train_step += 1
    else:
        avg_loss /= train_step
    losses.append(avg_loss)
    if i%5 == 0:
        print('Epochs : {} Loss : {}'.format(i+1, avg_loss))
        

Epochs : 1 Loss : 1.3860665559768677
Epochs : 6 Loss : 1.993565320968628
Epochs : 11 Loss : 0.8498644828796387
Epochs : 16 Loss : 0.6414665579795837
Epochs : 21 Loss : 0.48716631531715393
Epochs : 26 Loss : 0.2787194848060608


In [47]:
np.argmax(model.predict(x_data),axis = -1)

array([1, 0, 1, 0], dtype=int64)

#### 15.1 입출력 시퀀스
1. **Sequence - To - Sequence**
    - 입력 시퀀스를 받아 그에 맞는 출력 시퀀스를 예측한다.
    - 예를 들면 N일치의 주식 가격을 입력하면 네트워크는 각 입력값보다 하루 앞선 시퀀스의 가격을 출력한다.
2. **Sequence - To - Vector**
    - 입력 시퀀스를 받아 그에 맞는 출력 시퀀스를 받는데, 마지막 벡터를 제외하고 모두 무시한다.
    - 예를 들면 영화 후기 데이터를 입력 받으면 네트워크는 그에 해당하는 감정 점수를 출력하게 된다.
3. **Vector -To - Sequence**
    - 각 타임 스텝에 하나의 입력 벡터를 반복해서 네트워크에 주입하고 하나의 시퀀스를 출력한다.
    - 예를 들어 이미지나 CNN의 출력을 입력 받아서 이미지에 대한 captionizing을 진행할 수 있는 것이다.
4. **Encoder - Decoder**
    - vector-to-sequence인 인코더 뒤에 sequence-to-vector인 디코더를 연결한다.
    - 에를 들면 한 언어의 문장을 인코더가 벡터로 바꾸어 주고 디코더가 이 벡터를 다른 언어의 문장으로 디코딩한다.

#### 1. 시계열(Time Series) 예측하기 
- 간단하게 generate_time_series라는 함수를 만들어서 가상의 시계열 데이터를 만들어 보고자 한다.  

**시계열 데이터는 입력특성이 일반적으로 [batch_size, time_steps, dimensionality]인데 예측하고 싶은 값이 하나면 dimensionality = 1이고 많으면 1보다 크다.**

In [62]:
def generate_time_series(batch_size, n_steps):
    f1, f2,off1,off2 = np.random.rand(4,batch_size,1)
    time = np.linspace(0,1,n_steps)
    series = 0.5*np.sin((time-off1)*(f1*10 +10))
    series += 0.2 * np.sin((time-off2)*(f2*20+20))
    series += 0.1*(np.random.rand(batch_size, n_steps)-0.5)
    return series[:,:,np.newaxis].astype(np.float32)

In [63]:
n_steps = 50
series = generate_time_series(10000, n_steps = n_steps+1)
x_train, y_train = series[:7000, :n_steps], series[:7000, -1]
x_val, y_val = series[7000:9000, :n_steps], series[7000:9000, -1]
x_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [67]:
x_train.shape, x_test.shape

((7000, 50, 1), (1000, 50, 1))

In [68]:
#간단한 기준 성능을 설정해 주면 나중에 더 학습의 성능을 평가하기에 좋다.
#간단한 완전 연결 네트워크를 이용하면 좋다.
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = [50,1]))
model.add(tf.keras.layers.Dense(1))

model.compile(loss = 'mse', optimizer = 'adam')
model.fit(x_train, y_train, epochs = 20, verbose = 0)

In [71]:
model.evaluate(x_test,y_test)

1000/1000 [==============================] - 0s 113us/sample - loss: 0.0039


0.003947819799184799

- SimpleRNN에서는 각 RNN층의 모든 시퀀스 분석이 동시에 일어난다.
- 원래 시계열 분석을 할 때에는 마지막 출력값에만 관심이 있기 때문에 반환해 주는 값이 마지막 타임 스텝의 출력이다.
- 그러나 multi RNN layer을 쌓는 경우에는 그렇게 되면 다음 학습을 할때 2D의 입력값이 전해지기 때문에 반드시 return_sequences = True로 모든 타임 스텝의 예측값을 다음 layer에 전해 주어야 한다.

In [76]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.SimpleRNN(units = 20, return_sequences = True, input_shape = [None, 1]))
model.add(tf.keras.layers.SimpleRNN(units = 20))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_11 (SimpleRNN)    (None, None, 20)          440       
_________________________________________________________________
simple_rnn_12 (SimpleRNN)    (None, 20)                820       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 21        
Total params: 1,281
Trainable params: 1,281
Non-trainable params: 0
_________________________________________________________________


- 사실 SimpleRNN은 기본적으로 tanh를 activation function으로 갖기 때문에 -1에서 1사이의 값을 출력값으로 갖는다. 그렇기 때문에 예측된 값의 범위를 바꾸어 주기 위해 마지막 층은 Dense를 이용해서 출력을 하곤 한다.
- 아니면 직접 RNN layer을 customization을 해서 생성할 수도 있다.
- 그리고 항상 마지막이 되는 RNN layer에서의 return_sequences = True는 없애 주어야 한다.

In [79]:
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs =5, batch_size = 50)

Train on 7000 samples, validate on 2000 samples
Epoch 1/5
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0037 - accuracy: 0.0000e+00 - val_loss: 0.0035 - val_accuracy: 0.0000e+00
Epoch 2/5
7000/7000 [==============================] - 6s 850us/sample - loss: 0.0031 - accuracy: 0.0000e+00 - val_loss: 0.0033 - val_accuracy: 0.0000e+00
Epoch 3/5
7000/7000 [==============================] - 6s 908us/sample - loss: 0.0032 - accuracy: 0.0000e+00 - val_loss: 0.0029 - val_accuracy: 0.0000e+00
Epoch 4/5
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0030 - accuracy: 0.0000e+00 - val_loss: 0.0029 - val_accuracy: 0.0000e+00
Epoch 5/5
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0029 - accuracy: 0.0000e+00 - val_loss: 0.0030 - val_accuracy: 0.0000e+00


In [80]:
model.evaluate(x_test, y_test)

1000/1000 [==============================] - 1s 533us/sample - loss: 0.0029 - accuracy: 0.0000e+00


[0.002904352083802223, 0.0]

#### 2. Multi Step을 예측하기
- 지금까지는 시퀀스를 주고 하나의 벡터만 예측하는 것이었다면 이번에는 시퀀스를 받고 하나의 시퀀스를 또 예측하는 것이다.
- 이번에는 아까 만든 series를 이용해서 10개의 timestep앞을 예측하는 것을 시도해 보고자 한다.
- input_shape는 마지막 숫자가 단변량 데이터이기 떄문에 1이다. 즉, 각각의 타임 스텝마다 하나의 변수에 대한 값만 존재하기 때문이다.

In [81]:
series = generate_time_series(10000, n_steps + 10)
x_train, y_train = series[:7000, :n_steps], series[:7000, n_steps:]
x_val, y_val = series[7000:9000, :n_steps], series[7000:9000, n_steps:]
x_test, y_test = series[9000:, :n_steps], series[9000:, n_steps:]

In [82]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.SimpleRNN(units = 20, return_sequences = True, input_shape = [None, 1]))
model.add(tf.keras.layers.SimpleRNN(units = 20))
model.add(tf.keras.layers.Dense(10))
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_13 (SimpleRNN)    (None, None, 20)          440       
_________________________________________________________________
simple_rnn_14 (SimpleRNN)    (None, 20)                820       
_________________________________________________________________
dense_9 (Dense)              (None, 10)                210       
Total params: 1,470
Trainable params: 1,470
Non-trainable params: 0
_________________________________________________________________


- 그러나 위와 같은 방법으로 시도하게 되면 마지막 layer만 10개의 timestep을 예측하는 꼴이 된다.
- 따라서 우리는 모든 layer들이 10개씩 예측하도록 model의 설계를 바꿔 주어야 한다.즉, 각 time step마다 10차원의 벡터씩 예측하는 것이다.
    - 구체적인 예로 설명 하자면 첫번째 timestep이 입력되면 2-11까지예측하는 것이다.  
    
- sequence-to-sequence model로 바꾸기 위해서는 마지막 층을 포함한 모든 층에서 return_sequences = True여야 하고, 모든 타임 스텝에서 출력을 Dense층에 적용해야 한다.
    
- 이를 수행하기 위해 **TimeDistributed** layer을 이용하고, 이 층은 다른 층을 감싸서 입력 시퀀스의 모든 타임 스텝에 return_seqeunces = True를 적용한다. 이를 위해 해당 층은 각 타임 스텝을 별개의 샘플처럼 다루도록 입력의 크기를 바꾼다.  

In [83]:
y = np.empty((10000, n_steps, 10))
for i in range(1, 10+1):
    y[:,:,i-1] = series[:, i:i+n_steps,0]
y_train = y[:7000]
y_val = y[7000:9000]
y_test = y[9000:]

In [86]:
#y 데이터는 batch_size 가 10000이고 50개의 timestep, 그리고 10칸 앞의 값에 대해 예측을 한다.
y.shape

(10000, 50, 10)

In [88]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.SimpleRNN(units = 20, input_shape = [None, 1], return_sequences = True))
model.add(tf.keras.layers.SimpleRNN(units = 20, return_sequences = True))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(10)))

model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_17 (SimpleRNN)    (None, None, 20)          440       
_________________________________________________________________
simple_rnn_18 (SimpleRNN)    (None, None, 20)          820       
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 10)          210       
Total params: 1,470
Trainable params: 1,470
Non-trainable params: 0
_________________________________________________________________


- 그러나 문제는 이러한 간단한 SimpleRNN layer을 사용하는 것은 긴 시퀀스나 시계열에서는 잘 작동하지 않는다는 것이다.
- 이를 해결하기 위한 방법으로 LSTM과 GRU가 있다.